# Import lib

In [1]:
import requests
import pandas as pd
import folium
import time
import random
import matplotlib.pyplot as plt
import datetime

In [2]:
#1 choisir une ville par numero de departement
#2 afficher la population de la ville comme un fun fact (avec une ampoule à coté)
#3 récupérer la liste des piezomètres avec l'opération 'station'
#4 afficher la position des n piézomètre
#5 Selectionner un piezomètre
#6 afficher les courbe de la chronique avec l'opération 'chronique'

# Fonctions

In [3]:
def recup_commune(departement):
    #addresse api
    curl = f'https://geo.api.gouv.fr/communes?codePostal={str(departement)}'
    #requete api
    commune_req = requests.get(curl).json()
    
    #json
    commune = commune_req[0]
    
    #info
    insee = commune['code']
    ville = commune['nom']
    population = commune['population']
    
    print("{} possède une population de {:,.0f} habitants".format(ville, population))
    
    return insee, ville, population

In [4]:
def recup_list_stations(insee, ville):
    #code de la ville
    insee_str = str(insee)
    #url pour la requete
    station_url = f'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_commune={insee_str}&format=json&size=20&pretty'
    station_req = requests.get(station_url).json()
    #list des stations piezometriques
    stations = station_req['data']

    print(f"il y a {len(stations)} station(s) à {ville}")
    
    return station_req, stations

In [5]:
def print_station_date(station):
    #information pour être print
    n_mesures = station['nb_mesures_piezo']
    date_debut = station['date_debut_mesure']
    date_fin = station['date_fin_mesure']
    
    #recupère date sous format Timestamp
    date_debut = pd.to_datetime(date_debut)
    date_fin = pd.to_datetime(date_fin)
    
    #calcul deltat 
    delta_t = date_fin - date_debut
    delta_t_an = delta_t.days / 365
    delta_t_mois = delta_t.days / 30
    
    #mise en forme
    delta_t_str = "{:.2f}".format(delta_t_an)
    moy = n_mesures/(delta_t_mois) #permet d'avoir une idée de la résolution temporelle du graphique
    moy_str = "{:.2f} mesures/mois".format(moy)
    
    #mise en forme des dates debut et fin
    date_debut_str = date_debut.strftime("%Y-%m-%d")
    date_fin_str = date_fin.strftime("%Y-%m-%d")
    
    print(f"il y a {n_mesures} mesures | {date_debut_str} -> {date_fin_str} ({delta_t_str} ans) | moyenne = {moy_str}")

In [6]:
def random_station(stations):
    n_stations = len(stations) - 1
    idx = random.randint(0,n_stations)

    station = stations[idx]
    
    print_station_date(station)
    
    return station

In [7]:
def recup_chronique(station, size=1000):
    code_bss = str(station['code_bss'])
    
    chronique_url = f'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss}&size={str(size)}'

    chronique_req = requests.get(chronique_url).json()
    chroniques = chronique_req['data']
    print(f"il y a {len(chroniques)} mesure(s)")
    
    return chronique_req, chroniques

# Main

In [20]:
#1 choisir une ville par numero de departement
#2 afficher la population de la ville comme un fun fact (avec une ampoule à coté)
#3 récupérer la liste des piezomètres avec l'opération 'station'
#4 afficher la position des n piézomètre
#5 Selectionner un piezomètre
#6 afficher les courbe de la chronique avec l'opération 'chronique'

departement = 67202

insee, ville, population = recup_commune(departement)

station_req, stations = recup_list_stations(insee, ville)

station = random_station(stations)

chronique_req, chroniques = recup_chronique(station, size=2000)



Wolfisheim possède une population de 4,174 habitants
il y a 2 station(s) à Wolfisheim
il y a 291 mesures | 1967-08-01 -> 1998-01-28 (30.52 ans) | moyenne = 0.78 mesures/mois
il y a 291 mesure(s)
